# Deep Neural Net for Sentiment Classification

This Deep Net performs sentiment analysis on the IMDB review dataset.

In [ ]:
import os

import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from keras_contrib.layers.advanced_activations import SineReLU

import sklearn.metrics
from sklearn.metrics import roc_auc_score

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/dense'

epochs = 5
batch_size = 64
patience = 10
val_split = .3

n_dim = 128
n_unique_words = 20000
max_review_length = 200
pad_type = trunc_type = 'pre'

n_dense = 64
dropout = .5

#### Load Data

In [ ]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

In [ ]:
for x in X_train[0:6]:
    print(len(x))

In [ ]:
y_train[0:6]

In [ ]:
len(X_train), len(y_train)

#### Restore Words from index

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
word_index = {k : (v+3) for k,v in word_index.items()}
word_index['PAD'] = 0
word_index['START'] = 1
word_index['UNK'] = 2

In [ ]:
word_index

In [ ]:
index_word = {v:k for k,v in word_index.items()}

In [ ]:
X_train[0]

In [ ]:
' '.join(index_word[id] for id in X_train[0])

In [ ]:
(all_X_train, _), (all_X_valid, _) = imdb.load_data()

In [ ]:
' '.join(index_word[id] for id in all_X_train[0])

#### Preprocess Data

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
for x in X_train[0:6]:
    print(len(x))

In [ ]:
' '.join(index_word[id] for id in X_train[0])

In [ ]:
' '.join(index_word[id] for id in X_train[5])

#### Design Deep Net Architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(Flatten())
model.add(Dense(n_dense))
model.add(SineReLU(0.0095))

model.add(Dense(n_dense))
model.add(SineReLU(0.0095))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# Embedding 1
n_dim, n_unique_words, n_dim*n_unique_words

In [ ]:
# Flatten 1
max_review_length, n_dim, n_dim*max_review_length

In [ ]:
# Dense 1 (Fully Connected)
n_dense, n_dim*max_review_length*n_dense + n_dense

In [ ]:
# Dense 2
n_dense + 1

#### Configure the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath=output_dir + '/best-imdb-deep-net-model.hdf5', save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=patience)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### TensorBoard

In [ ]:
tensorboard = TensorBoard("../logs/imdb-deep-net-relus")

### Train the Model

In [ ]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=val_split, callbacks=[modelCheckpoint, earlyStopping])#, tensorboard])

#### Evaluate

In [ ]:
model = keras.models.load_model(output_dir + '/best-imdb-deep-net-model.hdf5', custom_objects={'SineReLU': SineReLU})

In [ ]:
y_hat = model.predict_proba(X_valid)
final_loss, final_acc = model.evaluate(X_valid, y_valid, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat) * 100

In [ ]:
print('{:0.2f}'.format(pct_auc))
print(np.std(history.history['loss']))

In [ ]:
fpr, tpr, _ = sklearn.metrics.roc_curve(y_valid, y_hat)
roc_auc = sklearn.metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

plt.show()

In [ ]:
#SineReLU V2



#ReLU



In [ ]:
# ydf = pd.DataFrame(list(zip(y_hat[:,0], y_valid)), columns=['y_hat', 'y'])
# ydf.head(10)

In [ ]:
# ' '.join(index_word[id] for id in all_X_valid[0])

In [ ]:
# ' '.join(index_word[id] for id in all_X_valid[6])

In [ ]:
# ydf[(ydf.y == 0) & (ydf.y_hat > 0.9)].head(10)

In [ ]:
# ' '.join(index_word[id] for id in all_X_valid[112])

In [ ]:
# ydf[(ydf.y == 1) & (ydf.y_hat < 0.1)].head(10)

In [ ]:
# ' '.join(index_word[id] for id in all_X_valid[101])